In [1]:
import json
import pandas as pd

In [21]:
with open('/home/pding/OneDrive/kph/fullms.json') as f:
  arto = json.load(f)

In [3]:
artpd = pd.read_json('/home/pding/OneDrive/kph/fullms.json', orient='index', convert_dates=False, convert_axes=False)

In [8]:
artpd = artpd.replace('\n',' ', regex=True)
artpd = artpd.replace('\t',' ', regex=True)

In [10]:
import spacy
from spacy.matcher import PhraseMatcher
import en_core_web_sm
svoc = en_core_web_sm.load()

In [14]:
artpda = artpd[artpd.abstract.notnull()]
artpda.index = pd.Series(artpda.index).apply(lambda x: x[0:8])
artpdak = artpda[artpda.keywords.str.len() > 0]
dataf = pd.DataFrame(index=artpdak.index, columns=['SRC', 'TRG', 'keywords', 'Extracted', 'abskey'])
dataf.loc[:, 'SRC'] = artpdak.title + ' ' + artpdak.abstract
dataf.loc[:, 'keywords'] = artpdak.keywords
matcher = PhraseMatcher(svoc.vocab, attr="LOWER")
for pmid in dataf.index:
    t0 = dataf.loc[pmid]
    patterns = [svoc.make_doc(str(name)) for name in t0.keywords]
    matcher.add("Names", None, *patterns)
    doc = svoc(t0.SRC)
    t1 = ['O']*(len(doc))
    matched = []
    matn = 0
    for _, start, end in matcher(doc):
        t1[start] = 'B'
        t1[start+1:end] = 'I'*(end-start-1)
        if str(doc[start:end]).lower() not in matched:
            matn = matn+1
            matched.append(str(doc[start:end]).lower())
    abskw = []
    for x in t0.keywords:
        try:
            if x.lower() not in matched:
                abskw.append(x)
        except(AttributeError, TypeError):
            pass
    dataf.loc[pmid, 'TRG'] = ' '.join([t for t in t1])
    dataf.loc[pmid, 'Extracted'] = matn
    dataf.loc[pmid, 'abskey'] = abskw
    matcher.remove("Names")
dataf.to_pickle("/home/pding/OneDrive/kph/processed3.pkl")

In [15]:
dataf

,SRC,TRG,keywords,Extracted,abskey
32460418,Chromatin remodelers in oligodendroglia. Oligo...,B I O O O O O O O O O O O O O O O O O O O O O ...,"[CNS development, chromatin remodelers, oligod...",1,"[CNS development, oligodendrogenesis, transcri..."
32460167,Clinical implications of serum neurofilament i...,O O O O O O O O O O O O O O O O O O O O O O B ...,"[Biomarker, Multiple sclerosis, Neurofilament ...",4,[Prediction]
32460145,Effect of cognitive rehabilitation on neuropsy...,O O B I O O O O O O O O O O O B I O O O O O O ...,"[Clinical trial, Cognition, Cognitive rehabili...",4,"[Clinical trial, Cognition]"
32460086,B-cell depleting therapies may affect suscepti...,B I I I I O O O O O O O O O O B I O O O B O O ...,"[B-cell depleting therapies, COVID-19, DMTs, M...",4,[]
32459513,The dichotomous role of the gut microbiome in ...,O O O O O B I O O O O B I O O O B I O O O O O ...,"[Alzheimer’s disease, Parkinson’s disease, gut...",5,"[Alzheimer’s disease, Parkinson’s disease, gut..."
...,...,...,...,...,...
13106716,Multiple sclerosis; its frequency and distribu...,O O O O O O O O O O O O O O O O O O O O O O O ...,[MULTIPLE SCLEROSIS/epidemiology],0,[MULTIPLE SCLEROSIS/epidemiology]
13106715,The allergic aspects of multiple sclerosis. Fo...,O O O O O O O O O O O O O O O O O O O O O O O ...,[ALLERGY/manifestations],0,[ALLERGY/manifestations]
13106714,Histamine treatment of multiple sclerosis. His...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[HISTAMINE/therapeutic use, MULTIPLE SCLEROSIS...",0,"[HISTAMINE/therapeutic use, MULTIPLE SCLEROSIS..."
14832406,Studies on acute disseminated encephalomyeliti...,O O O O B O O O O O O O O O O O O O O O O O O ...,"[CEREBROSPINAL FLUID, ENCEPHALOMYELITIS]",2,[]


In [13]:
dataf.loc[pmid]

SRC          The Relationships between Physical Activity, S...
TRG                                                        NaN
keywords     [None, efficacy expectations, exercise, health...
Extracted                                                  NaN
abskey                                                     NaN
Name: 31766489, dtype: object

In [4]:
artpda = artpd[artpd.abstract.notnull()]

In [5]:
artpda.index = pd.Series(artpda.index).apply(lambda x: x[0:8])

In [6]:
artpdak = artpda[artpda.keywords.str.len() > 0]

In [7]:
dataf = pd.DataFrame(index=artpdak.index, columns=['SRC', 'TRG', 'keywords', 'Extracted', 'abskey'])
dataf.loc[:,'SRC'] = artpdak.title + ' '+ artpdak.abstract

In [8]:
dataf.loc[:,'keywords'] = artpdak.keywords

In [9]:
import spacy
from spacy.matcher import PhraseMatcher
import en_core_web_sm
svoc = en_core_web_sm.load()

In [10]:
matcher = PhraseMatcher(svoc.vocab, attr="LOWER")

In [11]:
for pmid in dataf.index:
    t0 = dataf.loc[pmid]
    patterns = [svoc.make_doc(str(name)) for name in t0.keywords ]
    #matcher = PhraseMatcher(svoc.vocab, attr="LOWER")
    matcher.add("Names", None, *patterns)
    doc = svoc(t0.SRC)
    t1 = ['O']*(len(doc))
    matched = []
    matn = 0
    for _, start, end in matcher(doc):
        t1[start] = 'B'
        t1[start+1:end] = 'I'*(end-start-1)
        #print(doc[start:end])
        if str(doc[start:end]).lower() not in matched:
            matn = matn+1
            matched.append(str(doc[start:end]).lower())
    abskw = []
    for x in t0.keywords:
        if x.lower() not in matched:
            abskw.append(x)
    dataf.loc[pmid,'TRG'] = ' '.join([t for t in t1])
    dataf.loc[pmid,'Extracted'] = matn
    dataf.loc[pmid,'abskey'] = abskw
    matcher.remove("Names")

AttributeError: 'spacy.matcher.phrasematcher.PhraseMatcher' object has no attribute 'remove'

In [11]:
pmid = '32460418'

In [12]:
t0 = dataf.loc[pmid]
patterns = [svoc.make_doc(str(name)) for name in t0.keywords ]
#matcher = PhraseMatcher(svoc.vocab, attr="LOWER")
matcher.add("Names", None, *patterns)
doc = svoc(t0.SRC)
t1 = ['O']*(len(doc))
matched = []
matn = 0
for _, start, end in matcher(doc):
    t1[start] = 'B'
    t1[start+1:end] = 'I'*(end-start-1)
    #print(doc[start:end])
    if str(doc[start:end]).lower() not in matched:
        matn = matn+1
        matched.append(str(doc[start:end]).lower())
abskw = []
for x in t0.keywords:
    if x.lower() not in matched:
        abskw.append(x)
dataf.loc[pmid,'TRG'] = ' '.join([t for t in t1])
dataf.loc[pmid,'Extracted'] = matn
dataf.loc[pmid,'abskey'] = abskw
matcher.remove("Names")

In [13]:
dataf

,SRC,TRG,keywords,Extracted,abskey
32460418,Chromatin remodelers in oligodendroglia. Oligo...,B I O O O O O O O O O O O O O O O O O O O O O ...,"[CNS development, chromatin remodelers, oligod...",1,"[CNS development, oligodendrogenesis, transcri..."
32460167,Clinical implications of serum neurofilament i...,NaN,"[Biomarker, Multiple sclerosis, Neurofilament ...",NaN,NaN
32460145,Effect of cognitive rehabilitation on neuropsy...,NaN,"[Clinical trial, Cognition, Cognitive rehabili...",NaN,NaN
32460086,B-cell depleting therapies may affect suscepti...,NaN,"[B-cell depleting therapies, COVID-19, DMTs, M...",NaN,NaN
32459513,The dichotomous role of the gut microbiome in ...,NaN,"[Alzheimer’s disease, Parkinson’s disease, gut...",NaN,NaN
...,...,...,...,...,...
13106716,Multiple sclerosis; its frequency and distribu...,NaN,[MULTIPLE SCLEROSIS/epidemiology],NaN,NaN
13106715,The allergic aspects of multiple sclerosis. Fo...,NaN,[ALLERGY/manifestations],NaN,NaN
13106714,Histamine treatment of multiple sclerosis. His...,NaN,"[HISTAMINE/therapeutic use, MULTIPLE SCLEROSIS...",NaN,NaN
14832406,Studies on acute disseminated encephalomyeliti...,NaN,"[CEREBROSPINAL FLUID, ENCEPHALOMYELITIS]",NaN,NaN


In [25]:
dataf.loc['32460418'].keywords

['CNS development',
 'chromatin remodelers',
 'oligodendrogenesis',
 'transcription factors',
 'transcription regulation']

In [18]:
dataf.to_pickle("/home/pding/OneDrive/kph/processed2.pkl")

In [210]:
dataf.to_pickle("./processed.pkl")

In [200]:
patterns = [svoc.make_doc(str(name)) for name in t0.keywords ]

In [190]:
t0 = dataf.loc[pmid]
patterns = [svoc.make_doc(name) for name in t0.keywords ]
#matcher = PhraseMatcher(svoc.vocab, attr="LOWER")
matcher.add("Names", None, *patterns)
doc = svoc(t0.SRC)
t1 = ['O']*(len(doc))
matched = []
matn = 0
for _, start, end in matcher(doc):
    t1[start] = 'B'
    t1[start+1:end] = 'I'*(end-start-1)
    print(len(t1))
    print(doc[start:end])
    if str(doc[start:end]).lower() not in matched:
        matn = matn+1
        matched.append(str(doc[start:end]).lower())
dataf.loc[pmid,'TRG'] = ' '.join([t for t in t1])
dataf.loc[pmid,'ext perc'] = matn
matcher.remove("Names")

184
treatment-sequencing model
184
relapsing-remitting multiple sclerosis
184
health technology assessment
184
health technology assessment
184
relapsing-remitting multiple sclerosis
184
treatment-sequencing model
184
treatment-sequencing model
184
treatment-sequencing model


In [191]:
' '.join([t for t in t1])

'O O O O O B I I I O B I I I I O O B I I O O O B I I O O O O O O O O O O O O O O O O O O B I I I I O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B I I I O O O O O O O O O O O O O O O O O O O O O O O O O B I I I O O O O O O O O O O O O O O O O O O O O O O O O O B I I I O O O O O O'

In [185]:
t0 = dataf.loc[pmid]
patterns = [svoc.make_doc(name) for name in t0.keywords ]
#matcher = PhraseMatcher(svoc.vocab, attr="LOWER")
matcher.add("Names", None, *patterns)
doc = svoc(t0.SRC)
t1 = ['O']*(len(doc))
matched = []
matn = 0

In [187]:
len(doc)

184

In [178]:
t0.keywords

['Alzheimer’s disease',
 'Parkinson’s disease',
 'gut dysbiosis',
 'gut microbiome',
 'neuro-resilience',
 'neurodegenerative disorders',
 'polyphenols',
 'probiotics',
 'synbiotics']

In [177]:
matn

5

In [ ]:
for match_id, start, end in matcher(doc):
    print("Matched based on lowercase token text:", doc[start:end])

In [151]:
for _, start, end in matcher(doc):
    t1[start] = 'B'
    t1[start+1:end] = 'I'
    print(doc[start:end])
    if matched != str(doc[start:end]).lower():
        matn = matn+1
        matched = str(doc[start:end]).lower()

treatment-sequencing model
relapsing-remitting multiple sclerosis
health technology assessment
health technology assessment
relapsing-remitting multiple sclerosis
treatment-sequencing model
treatment-sequencing model


IndexError: list assignment index out of range

In [126]:
test0 = dataf.iloc[0]
patterns = [svoc.make_doc(name) for name in test0.keywords ]
matcher.add("Names", None, *patterns)
doc = svoc(test0.SRC)
t1 = ['O']*len(doc)
matched = [None]
matn = 0
for _, start, end in matcher(doc):
    t1[start] = 'B'
    t1[start+1:end] = 'I'
    print(doc[start:end])
    if matched != str(doc[start:end]).lower():
        matn = matn+1
        matched = str(doc[start:end]).lower()
dataf.loc[pmid,'TRG'] = ' '.join([t for t in t1])
dataf.loc[pmid,'ext perc'] = matn

Chromatin remodelers
chromatin remodelers


In [131]:
' '.join([t for t in t1])

'B I O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B I O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O'

In [119]:
test0.keywords

['CNS development',
 'chromatin remodelers',
 'oligodendrogenesis',
 'transcription factors',
 'transcription regulation']

In [114]:
type(doc[start:end])

spacy.tokens.span.Span

In [107]:
test0

SRC         Chromatin remodelers in oligodendroglia. Oligo...
TRG                                                       NaN
keywords    [CNS development, chromatin remodelers, oligod...
ext perc                                                  NaN
Name: 32460418, dtype: object

In [ ]:
# combine title and abstract

In [52]:
matcher = PhraseMatcher(svoc.vocab, attr="LOWER")

In [66]:
patterns = [svoc.make_doc(name) for name in ["Angela Merkel", "Barack Obama", "N.Y."]]
matcher.add("Names", None, *patterns)

doc = svoc("angela merkel and us president barack Obama in N.Y.")
for match_id, start, end in matcher(doc):
    print("Matched based on lowercase token text:", doc[start:end])

Matched based on lowercase token text: angela merkel
Matched based on lowercase token text: barack Obama
Matched based on lowercase token text: N.Y.


In [92]:
doc2 = ['I']*len(doc)

In [93]:
doc2

['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']

In [ ]:
sp = spacy.load('en_core_web_sm')

In [57]:
matcher(doc)

[(10631222085860127603, 0, 2),
 (10631222085860127603, 5, 7),
 (10631222085860127603, 8, 9)]

In [74]:
from torchtext.data import Field, BucketIterator

In [ ]:
SRC = Field()